In [2]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--workers',type=int,default=8,help='Number of workers for dataloader')
parser.add_argument('batch_size',type=int,default=128)
parser.add_argument('n_class',type=int,default=27,help='Number of styles')
parser.add_argument('n_class',type=int,default=27,help='Number of styles')
parser.add_argument('image_size',type=int,default=64,help='Size of traning data')
parser.add_argument('nc',type=int,default=3,help='Number of channels')
parser.add_argument('nz',type=int,default=150,help='z latent vector')
parser.add_argument('G_h_size',type=int,default=64,help='Size of feature maps in generator')
parser.add_argument('D_h_size',type=int,default=32,help='Size of feature maps in discriminator')
parser.add_argument('num_epochs',type=int,default=75)
parser.add_argument('lr',type=float,default=0.0001)
parser.add_argument('beta1 = 0.5',type=float,default=0.5,help='beta1 for adam')
parser.add_argument('ngpu',type=int,default=1)

_StoreAction(option_strings=[], dest='ngpu', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, help=None, metavar=None)

In [3]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') !=-1:
        nn.init.normal_(m.weights.data,0.0,0.02)
    elif classname.find('BatchNorm') !=-1:
        nn.init.normal_(m.weight.data,1.0,0.02)
        nn.init.constant_(m.bias.data,0)
        

        
# DCGAN generator
class DCGAN_G(nn.Module):
    def __init__(self,ngpu):
        super(DCGAN_G,self).__init__()
        self.ngpu = ngpu
        main = torch.nn.Sequential()
        
        
        # We need to know how many layers we will use at the beginning
        mult = param.image_size // 8
        
        
        ### Start block
        # Z_size random numbers
        main.add_module('Start-ConvTranspose2d',torch.nn.ConvTranspose2d(param.nz,param.G_h_size*mult,kernel_size=4,stride=1,padding=0,Bias=False))
        if param.SELU:
            main.add_module('Start-SELU',torch.nn.SELU(inplace=True))
        else:
            main.add_module('Start-BatchNorm2d',torch.nn.Batch2d(param.G_h_size*mult))
            main.add_module('Start-ReLU',torch.nn.ReLU())
        # Size = (G_h_size * mult)x 4 x4
        
        ### Middle block (Done until we reach  ? x image_size/2 x image_size/2)
        i=1
        while mult>1:
            main.add_module('Middle-ConvTranspose2d [%d]'%i, torch.nn.ConvTranspose2d(param.G_h_size*mult,param.G_h_size*(mult//2),kernel_size=4,stride=2,padding=1,bias=False))
            if param.SELU:
                main.add_module('Middle-SELU [%d]'%i,torch.nn.SELU(inplace=True))
            else:
                main.add_module('Middle-BatchNorm2d [%d]'%i,torch.nn.BatchNorm2d(param.G_h_size*(mult//2)))
                main.add_module('Middle_ReLU [$d]'%i,torch.nn.ReLU(inplace=True))
            # Size = (G_h_size * (mult/(2*i)))x8x8
            mult = mult // 2
            i+=1
        
        ### End block
        # Size = G_h_size/2 x image_size/2
        main.add_module('End-ConvTransposed2d', torch.nn.ConvTranspose2d(param.G_h_size,param.n_colors,kernel_size=4,stride=2,padding=1,bias=False))
        main.add_module('End-Tanh',torch.nn.Tanh())
        # Size = n_colors x image_size x image_size
        self.main = main
    
    def forward(self,input):
        if isinstance(input.data,torch.cuda.FloatTensor) and param.n_gpu>1:
            output = torch.nn.parallel.data_parallel(self.main,input,range(param.n_gpu))
        else:
            output = self.main(input)
        return output
    
# DCGAN discriminator ( using somewhat the reverse of the generator)
class DCGAN_D(torch.nn.Module):
    def __init__(self,ngpu):
        super(DCGAN_D,self).__init__()
        main = torch.nn.Sequential()
        self.ngpu = ngpu
        
        ### start block
        # size = n_colors x image_size x image_size
        main.add_module('Start-conv2d',torch.nn.Conv2d(param.n_colors,param.D_h_size,kernel=4,stride=2,padding=1,bias=False))
        if param.SELU:
            main.add_module('Start-SELU',torch.nn.SELU(inplace=True))
        else:
            main.add_module('Start-LeakyReLU',torch.nn.LeakyReLU(0.2,inplace=True))
        image_size_new = para.image_size // 2
        # Size = D_h_size x image_size/2 x image_size/2
        
        ### Middle block (Done until we reach ? x 4 x4)
        mult = 1
        i=0
        while image_size_new>4:
            main.add_module('Middle-Conv2d [%d]'%i,torch.nn.Conv2d(param.D_h_size*mult,param.D_h_size*(2*mult),kernel_size=4,stride=2,padding=1,biase=False))
            if param.SELU:
                main.add_module('Middle-SELU [%d]'%i,torch.nn.SELU(inplace=True))
            else:
                main.add_module('Middle-BatchNorm2d [%d]'%i,torch.nn.BatchNorm2d(param.D_h_size*(2*mult)))
                main.add_module('Middle-LeakyReLU [%d]'%i,torch.nn.LeakyReLU(0.2,inplace=True))
            # Size = (D_h_size*(2*i)) x image_size/(2*i) x image_size/(2*i)
            image_size_new = image_size_new // 2
            mult *=2
            i +=1
            
            ## End Block
            # size = (D_h_size*mult) x 4 x4
            main.add_module('End-Conv2d',torch.nn.Conv2d(param.D_h_size*mult,1,kernel_size=4,stride=1,padding=0,bias=False))
            main.add_module('Sigmoid',torch.nn.Sigmoid())
            # size = 1 x1 x1 (Is a real cat or not)
            self.main = main
            self.mult = mult
            
            self.discriminate = nn.Sequential(
                nn.Conv2d(param.D_h_size*mult,1,kernel_size=4,stride=1,padding=0,bias=False),
                nn.Sigmoid()
            )
            
            # test these two setting
            self.classify = nn.Sequential(
                nn.Conv2d(param.D_h_size*mult,n_class,kernel_size=4,stride=1,padding=0,bias=False)
            )
#             self.classify = nn.Sequential(
#                 nn.Linear(param.D_h_size*mult*4*4,1024),
#                 nn.LeakyReLU(0.2,inplace=True),
#                 nn.Linear(1024,512),
#                 nn.LeakyReLU(0.2,inplace=True),
#                 nn.Linear(512,n_class),
#                 nn.Softmax(dim=1)
#             )
            
        def forward(self,input):
            if isinstance(input.data,torch.cuda.FloatTensor) and param.n_gpu>1:
                output = torch.nn.parallel.data_parallel(self.main,input,range(param.n_gpu))
                d_out = torch.nn.parallel.data_parallel(self.discriminate,output,range(param.n_gpu))
                c_out = torch.nn.parallel.data_parallel(self.classify,output,range(param.n_gpu))
                # second option
                d_out = torch.nn.parallel.data_parallel(self.discriminate,output,range(param.n_gpu)).view(-1,1)
                c_out = torch.nn.parallel.data_parallel(self.classify,output.view(-1,param.D_h_size*self.mult*4*4),range(param.n_gpu))
            else:
                output = self.main(input)
                d_out = self.discriminate(output)
                c_out = self.classify(output)
                # second option
                d_out = self.discriminate(output).view(-1,1)
                c_out = self.classify(output.view(-1,param.D_h_size*self.mult*4*4))
            
            # Convert from 1 x 1 x 1 to 1 so that we can compare to given label (cat or not?)
            return d_out,c_out


In [4]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from dataloader_wikiart import *
from model_style import * 

manualSeed = 999
print("Random Seed: ",manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

# Create the dataloader
# dataset = StyleDataset()
# dataloader = DataLoader(dataset,batch_size,shuffle=True,num_workers=workers,pin_memory=True)
dataloader = get_dataset()

# Dataloader usage in training loop:
# for i,(data,target,label) in enumerate(dataloader): data=image, target=onehot encoded style,label=style

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

ModuleNotFoundError: No module named 'dataloader_wikiart'

In [5]:
# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

NameError: name 'dataloader' is not defined

In [6]:
# Create the generator
netG = Generator(ngpu).to(device)

# # Handle multi-gpu if desired
# if (device.type == 'cuda') and (ngpu > 1):
#     netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)

NameError: name 'Generator' is not defined

In [7]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# # Handle multi-gpu if desired
# if (device.type == 'cuda') and (ngpu > 1):
#     netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)

NameError: name 'Discriminator' is not defined

In [8]:
criterion = nn.BCELoss()
criterion_style = nn.CrossEntropLoss()

## create batch of latent vectors that we will use to visualize
# the progression of the generator
fixed_noise = torch.randn(64,param.nz,1,1,device=device)

# establish convention for real and fake labels during training
real_label = 1.
fake_label = 0

optimizerD = optim.Adam(netD.parameters(),lr=lr,betas=(beta1,0.999))
optimizerG = optim.Adam(netG.parameters(),lr=lr,betas=(beta1,0.999))

AttributeError: module 'torch.nn' has no attribute 'CrossEntropLoss'

In [1]:
# Training loop

# lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
entropies = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    for i,(data,style_label) in enumerate(dataloader,0):
        # update D: maximize log(D(x)) + log(1-D(G(z)))
        # Train with all-real batch
        netD.zero_grad()
        style_label = style_label.to(device)
        real_cpu = data.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,),real_label , dtype=torch.float,device=device)
        # Forward pass real batch through D
        output,output_style = netD(real_cpu)
        # Calculate loss on real
        errD_real = criterion(output,label)
        errD_real = errD_real + criterion_style(output_style.squeeze(),style_label.squeeze())
        # Calc gradients for D
        errD_real.backward()
        D_x = output.mean().item()
        
        ## Train with fake
        noise = torch.randn(b_size,param.nz,1,1,device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill(fake_label)
        #classify all fake image batch D
        output,output_style = netD(fake.detach())
        # Calculate D loss on fake
        errD_fake = criterion(output,label)
        # calc the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD=errD_real + errD_fake
        #update D
        optimizerD.step()
        
        # Update G: maximize log(D(G(z)))
        netG.zero_grad()
        label.fill_(real_label)# fake labels are real for generator cost
        output,output_style = netD(fake)
        # Uniform cross entropy
        logsoftmax=nn.LogSoftmax(dim=1)
        unif = torch.full((data.shape[0],n_class),1/n_class)
        unif = unif.to(device)
        # Calc G's loss based on this output
        
        errG = criterion(output,label)
        errG = errG + torch.mean(-torch.sum(unif*logsoftmax(output_style),1))
        # Calc gradients for 
        errG.backward()
        D_G_z2 = output.mean().item()
        #Update G
        optimizerG.step()
        
        style_entropy = -1*(nn.functional.softmax(output_style,dim=1)*nn.functional.log_softmax(out_style,dim=1))
        style_entropy = style_entropy.sum(dim=1).mean()/torch.log(torch.tensor(n_class).float())
        
        # output training stats
        if i%50==0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f\t Entropy: %.4f'
                 %(epoch,num_epochs,i,len(dataloader),
                  errD.item(),errG.item(),D_x,D_G_z1,D_G_z2,style_entropy))
            
        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        entropies.append(style_entropy)
        
        # Check how the generator is doing by saving G's output on fixed_noise
        if(iters%500==0) or ((epoch==num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake,padding=2, normalize=True))
            
        iters +=1

Starting Training Loop...


NameError: name 'num_epochs' is not defined